In [5]:
# Pissible need package install if needed

#Used for import function from other ipynb file
#pip install ipynb

#Install pytorch
#pip install pytorch

In [17]:
import os
import time
import copy
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
from ipynb.fs.full.utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug

In [18]:
mnist_data = "mnist_data"
mnist_result = "mnist_result"

if not os.path.exists(mnist_data):
    os.mkdir(mnist_data)

if not os.path.exists(mnist_result):
    os.mkdir(mnist_result)

cifar_data = "cifar_data"
cifar_result = "cifar_result"

if not os.path.exists(cifar_data):
    os.mkdir(cifar_data)

if not os.path.exists(cifar_result):
    os.mkdir(cifar_result)


In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [20]:
#load data
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset('MNIST',mnist_data)

In [21]:
images_all = []
labels_all = []
indices_class = [[] for c in range(num_classes)]

In [22]:
images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
labels_all = [dst_train[i][1] for i in range(len(dst_train))]

In [23]:
for i, lab in enumerate(labels_all):
    indices_class[lab].append(i)
images_all = torch.cat(images_all, dim=0).to(device)
labels_all = torch.tensor(labels_all, dtype=torch.long, device=device)

In [24]:
for c in range(num_classes):
    print('class c = %d: %d real images'%(c, len(indices_class[c])))

class c = 0: 5923 real images
class c = 1: 6742 real images
class c = 2: 5958 real images
class c = 3: 6131 real images
class c = 4: 5842 real images
class c = 5: 5421 real images
class c = 6: 5918 real images
class c = 7: 6265 real images
class c = 8: 5851 real images
class c = 9: 5949 real images


In [25]:
#Get the net
ConvNet = get_network('ConvNet',channel,num_classes,im_size).to(device)

In [30]:
#first train with dataset
def epoch(mode, dataloader, net, optimizer, criterion,device):
    loss_avg, acc_avg, num_exp = 0,0,0
    net = net.to(device)
    criterion = criterion.to(device)
    if mode == 'train':
        net.train()
    else:
        net.eval()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 10)
    for i_batch,datum in enumerate(dataloader):
        img = datum[0].float().to(device)
        lab = datum[1].long().to(device)
        n_b = lab.shape[0]
        
        output = net(img)
        loss = criterion(output,lab)
        acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
        
        loss_avg += loss.item()*n_b
        acc_avg += acc
        num_exp += n_b
        
        if mode == 'train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
    loss_avg /= num_exp
    acc_avg /= num_exp
    
    return loss_avg, acc_avg


def train_with_dataset(net, images_train, labels_train, testloader,device):
    net = net.to(device)
    images_train = images_train.to(device)
    labels_train = labels_train.to(device)
    lr = 0.01
    Epoch = 20
    optimizer = torch.optim.SGD(net.parameters(), lr = lr, momentum = 0.9, weight_decay = 0.0005)
    criterion = nn.CrossEntropyLoss().to(device)
    dst_train = TensorDataset(images_train,labels_train)
    trainloader = torch.utils.data.DataLoader(dst_train, batch_size = 256, shuffle = True, num_workers = 0)
    for ep in range(Epoch):
        loss_train, acc_train = epoch('train',trainloader, net, optimizer, criterion, device)
        print("loss train: {}, acc_trainL {}".format(loss_train, acc_train))
    loss_test, acc_test = epoch('test', testloader, net, optimizer, criterion, device)
    #print('%s Evaluate_%02d: epoch = %04d train time = %d s train loss = %.6f train acc = %.4f, test acc = %.4f' % (get_time(), it_eval, Epoch, int(time_train), loss_train, acc_train, acc_test))
    return acc_train, acc_test


In [31]:
ConvNet = get_network('ConvNet',channel,num_classes,im_size).to(device)
images, labels = copy.deepcopy(images_all.detach()), copy.deepcopy(labels_all.detach())

In [32]:
acc_train,acc_test = train_with_dataset(ConvNet,images, labels, testloader, device)
print("Acc train {}, Acc test {}".format(acc_train, acc_test))

loss train: 0.2845669314642747, acc_trainL 0.9305
loss train: 0.07055715382099152, acc_trainL 0.98095
loss train: 0.051819219532608984, acc_trainL 0.9856833333333334
loss train: 0.0431235013773044, acc_trainL 0.9882333333333333
loss train: 0.03930354802906513, acc_trainL 0.9890333333333333
loss train: 0.03349100859761238, acc_trainL 0.9910666666666667
loss train: 0.030719790793458622, acc_trainL 0.9916166666666667
loss train: 0.028566329753398897, acc_trainL 0.9921666666666666
loss train: 0.027640800115962823, acc_trainL 0.9923666666666666
loss train: 0.0252966020449996, acc_trainL 0.9929666666666667
loss train: 0.023900788811345896, acc_trainL 0.9934333333333333
loss train: 0.022544928916792075, acc_trainL 0.9939833333333333
loss train: 0.019872964514543612, acc_trainL 0.9947
loss train: 0.018982830706238747, acc_trainL 0.9950833333333333
loss train: 0.01840347923139731, acc_trainL 0.9953666666666666
loss train: 0.01773046632607778, acc_trainL 0.9957
loss train: 0.016535125508904458, 

NameError: name 'it_eval' is not defined